<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We use these libraries to collect historical market data, organize and store it efficiently, and work with dates and times. These tools have everything we need to analyze stock market performance over time.

In [ ]:
import datetime as dt

In [ ]:
import arcticdb as adb
import yfinance as yf

Here, we create a direct connection to a local database where we'll keep our financial data. Setting up this library once makes saving and reusing data much faster.

In [ ]:
arctic = adb.Arctic("lmdb://arcticdb_equity")
lib = arctic.get_library("demo", create_if_missing=True)

This setup initializes our data environment so we can easily store results and access them later. The code creates a workspace that holds our downloaded information, reducing the need for repeated downloads.

## Download and store market data

We define the time period to study, the daily frequency, and choose a mix of iconic global stocks and an index. We then fetch all the historical price data at once.

In [ ]:
start = dt.datetime(2013, 1, 1)
end = dt.datetime(2022, 12, 31)
freq = "1d"
symbols = {
    "^GSPC": "S&P 500",
    "AAPL": "Apple",
    "MSFT": "Microsoft",
    "GOOG": "Google",
    "AMZN": "Amazon",
    "NVDA": "Nvidia",
    "META": "Meta",
    "TSLA": "Tesla",
    "TSM": "TSMC",
    "TCEHY": "Tencent",
    "005930.KS": "Samsung",
    "ORCL": "Oracle",
    "ADBE": "Adobe",
    "ASML": "ASML",
    "CSCO": "Cisco",
}

In [ ]:
hist = yf.download(list(symbols.keys()), interval=freq, start=start, end=end)

This block pulls daily data for each chosen company and index from a reliable source covering nearly a decade. We define our universe upfront, making our analysis clear and repeatable.
By setting a consistent start and end date, we make sure we’re comparing all assets on equal footing. Storing the returned data means we can work efficiently with it later with no repeated downloads.

For each price type—like "Close" or "Open"—we save that data to our database. This means we can access any type of price history we want, organized for easy future use.

In [ ]:
for l in hist.columns.levels[0]:
    lib.write(l, hist[l])

We loop through different categories of price information and write each one to our data library. This step gives us flexibility to analyze any aspect of the market data later.
Because different stocks may have data formatted in slightly different ways, breaking it down by type ensures nothing gets missed. Writing these organized sections helps us avoid confusion later.

## Clean and process price data

We grab all historical closing prices, fill in any missing days with the latest available value, and check for gaps. We then calculate day-to-day returns and store them for analysis.

In [ ]:
hist_adj_close = lib.read("Close").data
hist_adj_close_clean = hist_adj_close.ffill()
hist_adj_close_clean.isnull().any().any()
hist_daily_returns = hist_adj_close_clean.pct_change(1).iloc[1:]
returns_sym = "hist/returns_Close_clean"
lib.write(returns_sym, hist_daily_returns)

This section ensures we have a complete, consistent record for every asset—even if some markets took holidays or missed a day. Filling missing spots with the most recent value keeps our measures real and avoids errors.
We measure performance as percentage change from one day to the next, a standard way to compare very different stocks. These new results are also saved, making future steps much faster.

## Calculate and visualize rolling betas

We focus on a more recent time window, then compare each stock’s day-to-day performance with the S&P 500. For each company, we calculate how much it tends to move in line with the market, updating our measure every six months or so.

In [ ]:
index_ticker = "^GSPC"
roll_days = 130
beta_start = dt.datetime(2018, 1, 1)
beta_end = dt.datetime(2022, 12, 31)
beta_returns = lib.read(returns_sym, date_range=(beta_start, beta_end)).data
index_returns = beta_returns[index_ticker]
stock_returns = beta_returns.drop(columns=index_ticker)

In [ ]:
rolling_cov = stock_returns.rolling(roll_days).cov(index_returns).iloc[roll_days - 1 :]
rolling_index_var = index_returns.rolling(roll_days).var().iloc[roll_days - 1 :]
rolling_beta = rolling_cov.divide(rolling_index_var, axis="index").rename(
    columns=symbols
)

In [ ]:
ax = rolling_beta.plot(grid=True, linewidth=0.9, title=f"Rolling {roll_days}-day beta")
ax.legend(loc="upper left")

By looking at performance in overlapping six-month periods, we see how each stock’s link to the wider market changes over time. We first measure how closely each one tracks the S&P 500, then turn this into an intuitive “beta” score.
A higher beta means a stock tends to move more than the market, while something near zero tracks it less closely. Finally, we plot these results together, letting patterns and differences stand out clearly. This gives us a practical view of risk and correlation across global leaders.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.